<a href="https://colab.research.google.com/github/cwinsor/medical_image_uw_madison/blob/main/colab_02_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
import os

In [2]:
assert ('google.colab' in sys.modules), "ERROR - the script expects to be run in Colab"

In [3]:
import torch
torch.cuda.is_available()

True

In [4]:
# we persist code, dataset and runs on google drive...
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
# we probably don't want to do a "git fetch" but a "--dry-run" may be helpful to check if there's repo updates
# versions are already set from the prior script colab_01 ..."
# so even if they are out-of-date we wouldn't want to update them here, we'd go
# back to the earlier script...

work_folder = '/content/gdrive/MyDrive/Colab_UW_Madison/medical_image_uw_madison/'
os.chdir(work_folder)
print("checking git status (git fetch --dry-run):")
!pwd
!git fetch --dry-run

checking git status (git fetch --dry-run):
/content/gdrive/MyDrive/Colab_UW_Madison/medical_image_uw_madison
From https://github.com/cwinsor/medical_image_uw_madison
   b19876f..c3ca8f4  main       -> origin/main


# Install dependencies

In [6]:
!apt update
# !apt install python3-venv python3-pip

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [731 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [108 kB]
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [734 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 1,802 kB in 3s (711 kB/s)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
8 packages can be upgraded. Run 'apt list --upgradable' to see them.


In [7]:
pip --version

pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)


In [8]:
!pip --version

pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)


In [9]:
run_folder = '/content/gdrive/MyDrive/Colab_UW_Madison/medical_image_uw_madison/project/Kaggle-UWMGIT'
os.chdir(run_folder)
!pwd

/content/gdrive/MyDrive/Colab_UW_Madison/medical_image_uw_madison/project/Kaggle-UWMGIT


In [10]:
!pip list > tempfile3a.txt

In [13]:
!pip install -r ../requirements_cw1b.txt  # NOTE this is "cw" requirements one level up

  Using cached mmcv-1.3.1-py2.py3-none-any.whl
  Using cached mmcv-full-1.3.1.tar.gz (259 kB)
  Preparing metadata (setup.py) ... done
  Using cached addict-2.4.0-py3-none-any.whl (3.8 kB)
  Using cached yapf-0.40.1-py3-none-any.whl (250 kB)
  Using cached importlib_metadata-6.8.0-py3-none-any.whl (22 kB)
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for mmcv-full
  Running setup.py clean for mmcv-full
Failed to build mmcv-full
ERROR: Could not build wheels for mmcv-full, which is required to install pyproject.toml-based projects


In [12]:
!pip list > tempfile3b.txt

# Add to PATH (if required)

## Comparing ***os.getenv('PATH')*** to ***sys.path***
* the former is path for the environment shell (applications)
* the latter is the python search path for modules. Importantly it is constructed when Python *IS STARTED* and is:
  * the folder of the python interpreter
  * installation-dependent standard folders
  * the contents of PYTHONPATH
  
in other words... updating PYTHONPATH won't help inside a jupyter notebook!


In [29]:
THE_FOLDER = "foo"
# THE_FOLDER = os.getcwd()

if THE_FOLDER in sys.path:
    print("already exists")
else:
    sys.path.append(THE_FOLDER)
    print("added")

already exists


# PyTorch Distributed...

https://pytorch.org/docs/stable/distributed.html

    How to use this module:

    Single-Node multi-process distributed training

    python -m torch.distributed.launch --nproc-per-node=NUM_GPUS_YOU_HAVE
            YOUR_TRAINING_SCRIPT.py (--arg1 --arg2 --arg3 and all other
            arguments of your training script)

    Multi-Node multi-process distributed training: (e.g. two nodes)

    Node 1: (IP: 192.168.1.1, and has a free port: 1234)
    python -m torch.distributed.launch --nproc-per-node=NUM_GPUS_YOU_HAVE
            --nnodes=2 --node-rank=0 --master-addr="192.168.1.1"
            --master-port=1234 YOUR_TRAINING_SCRIPT.py (--arg1 --arg2 --arg3
            and all other arguments of your training script)

    Node 2:
    python -m torch.distributed.launch --nproc-per-node=NUM_GPUS_YOU_HAVE
            --nnodes=2 --node-rank=1 --master-addr="192.168.1.1"
            --master-port=1234 YOUR_TRAINING_SCRIPT.py (--arg1 --arg2 --arg3
            and all other arguments of your training script)


# Ours is single-node multi-GPU.
So the shell command would be:

    cd /content/gdrive/MyDrive/Colab_UW_Madison/medical_image_uw_madison/project/Kaggle-UMWGIT

    CONFIG='work_configs/tract/final_solution/classification_configs/cls_1.py'
    NUM_GPUS_PER_NODE=2
    MASTER_PORT=1234
    ARGS='foo="bar" lala="foofoo"'

    python -m torch.distributed.launch --nproc-per-node=$NUM_GPUS_PER_NODE
            tools/train.py $CONFIG --launcher pytorch $ARGS
            

# let's do that now...

In [11]:
run_folder = '/content/gdrive/MyDrive/Colab_UW_Madison/medical_image_uw_madison/project/Kaggle-UWMGIT'
os.chdir(run_folder)
!pwd

/content/gdrive/MyDrive/Colab_UW_Madison/medical_image_uw_madison/project/Kaggle-UWMGIT


In [12]:
CONFIG='work_configs/tract/final_solution/classification_configs/cls_1.py'
NUM_GPUS_PER_NODE=1
MASTER_PORT=1234
ARGS='foo="bar" lala="foofoo"'

In [13]:
!python -m torch.distributed.launch --nproc-per-node=$NUM_GPUS_PER_NODE tools/train.py $CONFIG --launcher pytorch $ARGS

/usr/local/lib/python3.10/dist-packages/torch/distributed/launch.py:181: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use-env is set by default in torchrun.
If your script expects `--local-rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  warnings.warn(
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
Traceback (most recent call last):
  File "/content/gdrive/MyDrive/Colab_UW_Madison/medical_image_uw_madison/project/Kaggle-UWMGIT/tools/train.py", line 15, in <module>
    from mmseg import __version__
ModuleNotFoundError: No module named 'mmseg'
ERROR:torch.distributed.elastic.multiprocessing.api:failed (exitcode: 1) local_rank: 0 (pid: 9800) of binary: /usr/bin/python3
Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_mo